In [ ]:
import numpy as np
import os
import pandas as pd
import sys

from matplotlib import pyplot as plt
from tqdm import tqdm

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)
from experiments.MNIST import ExperimentMNISTBase, ExperimentADeLEn, ExperimentSupervised

In [ ]:
from itertools import chain
def generate_roc_df(roc_list:list) -> pd.DataFrame:
    ''' 
        Create a DataFrame from a list of roc curves
        Args:
        -----
            roc_list: list
                List of N roc curves where N is the number of iterations. It is a 
                list of tuples of the form (fpr, tpr), where fpr is the false positive
                rate and tpr is the true positive rate.
        Returns:
        --------
            roc_df: pd.DataFrame
                DataFrame with multiindex
    '''
    index_names = [
        list(map(lambda x: 'It {}'.format(x), np.repeat(np.arange(len(roc_list)), 2) + 1 )),
        ['FPR', 'TPR']*len(roc_list)
    ]
        
    tuples = list(zip(*index_names))
    index = pd.MultiIndex.from_tuples(tuples)
    roc_df = pd.DataFrame(chain.from_iterable(roc_list), index=index)
    return roc_df

def generate_multi_df(data:list, index_names:list) -> pd.DataFrame:
    index_names = [
        list(map(lambda x: 'It {}'.format(x), np.repeat(np.arange(len(data)), len(index_names)) + 1 )),
        index_names*len(data)
    ]

    tuples = list(zip(*index_names))
    index = pd.MultiIndex.from_tuples(tuples)
    return pd.DataFrame(chain.from_iterable(data), index=index)

def save_result(roc:list, scores:list, metrics:np.ndarray, config:dict) -> tuple:
    '''
        Save the results of the experiment
        Args:
        -----
            roc: list
                List of roc curves
            auc: list
                List of AUC scores
            config: dict
                Configuration of the experiment

        Returns:
        --------
            roc_df: pd.DataFrame
                ROC Cuve dataFrame with multiindex, considering the iterations.
            auc_df: pd.DataFrame
                DataFrame with the AUC scores
    '''

    roc_df = generate_multi_df(roc, ['FPR', 'TPR']).T
    scores_df = generate_multi_df(scores, ['Normal', 'Anomaly']).T
    metrics_df = pd.DataFrame(metrics, columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
    
    roc_df.to_pickle(os.path.join(config['save_result_dir'], 'roc.pkl'))
    scores_df.to_pickle(os.path.join(config['save_result_dir'], 'sample_score.pkl'))
    metrics_df.to_csv(os.path.join(config['save_result_dir'], 'metrics.csv'))

    return roc_df, scores_df, metrics_df

In [ ]:
from experiments.MNIST import ExperimentMNISTBase, ExperimentADeLEn

def n_labeled_test(ExperimentClass:ExperimentMNISTBase, d=None):
    n_iter = 25
    seed = 2*np.arange(n_iter, dtype=int) + 42

    pollution = 0.05
    known_anomalies_exp = [.01, .05, .2, .25]
    for known_anomalies in known_anomalies_exp:
        print(f'Exp. {known_anomalies=}')
        iterator = tqdm(
                    range(n_iter),
                    leave=True,
                    unit="It.",
                    postfix={"AUC": "%.3f" % -1},
                )

        roc, scores = [], []
        metrics = np.empty((n_iter, 5)) # acc, prec, rec, f1, auc

        for it in iterator:
            exp = ExperimentClass(known_anomalies=known_anomalies, pollution=pollution, d=d, seed=int(seed[it]))
            if it == 0:
                config = exp.config()
                exp.save_config()
            
            
            auc_score = exp.run()
            iterator.set_postfix({"AUC": "%.3f" % auc_score})

            fpr, tpr, roc_auc = exp.roc_curve()
            normal_scores, anomaly_scores = exp.score_per_label()
            acc, prec, rec, f1 = exp.classification_metrics()
            
            roc.append((fpr, tpr))   
            scores.append((normal_scores, anomaly_scores))
            metrics[it] = [acc, prec, rec, f1, roc_auc]
            
        roc_df, scores_df, metrics_df = save_result(roc, scores, metrics, config)
        
    return roc_df, scores_df, metrics_df

In [ ]:
bottleneck = [2, 5, 10]
for d in bottleneck:
    print(f'Exp. {d=}')
    roc_df, scores_df, metrics_df = n_labeled_test(ExperimentADeLEn, d=d)

# MedMNIST

In [ ]:
from experiments.MedMNIST import ExperimentMedMNISTBase, ExperimentADeLEn

def n_labeled_test(ExperimentClass:ExperimentMNISTBase, d=None):
    n_iter = 25
    seed = 2*np.arange(n_iter, dtype=int) + 42

    pollution = 0.05
    known_anomalies_exp = [.05, .2, .25, .5]
    for known_anomalies in known_anomalies_exp:
        print(f'Exp. {known_anomalies=}')
        iterator = tqdm(
                    range(n_iter),
                    leave=True,
                    unit="It.",
                    postfix={"AUC": "%.3f" % -1},
                )

        roc, scores = [], []
        metrics = np.empty((n_iter, 5)) # acc, prec, rec, f1, auc

        for it in iterator:
            exp = ExperimentClass(known_anomalies=known_anomalies, pollution=pollution, d=d, seed=int(seed[it]))
            if it == 0:
                config = exp.config()
                exp.save_config()
            
            
            auc_score = exp.run()
            iterator.set_postfix({"AUC": "%.3f" % auc_score})

            fpr, tpr, roc_auc = exp.roc_curve()
            normal_scores, anomaly_scores = exp.score_per_label()
            acc, prec, rec, f1 = exp.classification_metrics()
            
            roc.append((fpr, tpr))   
            scores.append((normal_scores, anomaly_scores))
            metrics[it] = [acc, prec, rec, f1, roc_auc]
            
        roc_df, scores_df, metrics_df = save_result(roc, scores, metrics, config)
        
    return roc_df, scores_df, metrics_df

In [ ]:
bottleneck = [2, 5, 10]
for d in bottleneck:
    print(f'Exp. {d=}')
    roc_df, scores_df, metrics_df = n_labeled_test(ExperimentADeLEn, d=d)

In [ ]:
a